In [32]:
import os
import sys
import collections
import csv
import pandas as pd
import numpy as np
import tensorflow as tf
import pandas as pd
import numpy as np
import time
from pymongo import MongoClient
import urllib
import multiprocess
import pickle
import random

# BERT files
os.listdir("../bert-master")
sys.path.insert(0, '../bert-master')

from run_classifier import *
import modeling
import optimization
import tokenization

import preprocessor as p


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
p.set_options(p.OPT.URL, p.OPT.EMOJI)

In [5]:
# Set up data directories
data_dir = './data'
model_dir = './model'
output_dir = './output'

if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
if not os.path.exists(model_dir):
    os.makedirs(data_dir)

In [6]:
username = 'christian'
password = 'Dec211996'

client = MongoClient('mongodb://' + urllib.parse.quote_plus(username) + ':' + urllib.parse.quote_plus(password) + '@198.211.115.252')

In [34]:
def worker(consp_tuple): 
    
    ## this is temp
    from pymongo import MongoClient
    import urllib
    import pickle
    username = 'christian'
    password = 'Dec211996'

    client = MongoClient('mongodb://' + urllib.parse.quote_plus(username) + ':' + urllib.parse.quote_plus(password) + '@198.211.115.252')
    
    data_dir = './data'
    ## end temp
    
    data = []
    
    conspiracy, hashtag = consp_tuple
    cursor = client[conspiracy][hashtag].find({})
    for i, document in enumerate(cursor):
        try:
            inputs = {'text' : p.clean(document['text'])}
            inputs.update({'tweetId' : document['tweetId']})
            data.append(inputs)
        except Exception as e:
            pass


        if (i > 100):
            break

    with open(data_dir + '/' + conspiracy + '-' + hashtag + '.p', 'wb') as f:
        pickle.dump(data, f)

    return consp_tuple

def get_consp_tuples():
    username = 'christian'
    password = 'Dec211996'

    client = MongoClient('mongodb://' + urllib.parse.quote_plus(username) + ':' + urllib.parse.quote_plus(password) + '@198.211.115.252')
    
    ignore_mask = ['test_db', 'admin', 'local', 'config', 'TwitterJobs']
    conspiracies = list(set(client.list_database_names()) - set(ignore_mask))

    consp_tuples = []
    for conspiracy in conspiracies:
        for hashtag in client[conspiracy].list_collection_names():
            consp_tuples.append((conspiracy, hashtag))

    return consp_tuples

def preprocess():
    consp_tuples = get_consp_tuples()
    random.shuffle(consp_tuples)
    p = multiprocess.Pool(multiprocess.cpu_count())
#     consp_tuples = [(consp_tuple, data_dir) for consp_tuple in consp_tuples]
#     results = p.map(worker, consp_tuples)

    for i in range(10):
        worker(consp_tuples[i])

preprocess()

In [27]:
def generate_labels():
    conspiracies = set()
    
    for filename in os.listdir(data_dir):
        if '.p' in filename:
            conspiracies.add(filename.split('-')[0])
    return {consp:i for i, consp in enumerate(conspiracies)}

In [58]:
data = []
dummytext = '*'

labels = generate_labels()

for filename in os.listdir(data_dir):
    if '.p' in filename:
        with open(data_dir + '/' + filename, 'rb') as f:
            consp = filename.split('-')[0]
            x = pickle.load(f)
            for row in x:
                data.append({'dummy_1': row['tweetId'], 'target': labels[consp], 'dummy_2' : dummytext, 'text' : row['text']})
df = pd.DataFrame(data)
data = None

                  dummy_1  target dummy_2  \
0              7290076266       4       *   
1              7438885529       4       *   
2              7478676096       4       *   
3              7466879468       4       *   
4              9391823778       4       *   
...                   ...     ...     ...   
2035  1271709878273392640       8       *   
2036  1271638554079301632       8       *   
2037  1273032402366091265       8       *   
2038  1272979618346102784       8       *   
2039  1272913869414023168       8       *   

                                                   text  
0     New Year Begins with Random Searches on the St...  
1     #PRISONPLANET "...contraband were #planted by ...  
2     Alex Jones has 3 new protest #videos for 2010 ...  
3     I do not believe I should give up my constitut...  
4     RT @vivos2012: #prisonplanet leaks secret abou...  
...                                                 ...  
2035  A friend from London just sent me this! It see.

In [79]:
# force train into cola format, test is fine as it is
# train = train.sample(frac=0.01)
# test = train.sample(frac=0.01)

test_split = 0.2

permuted_indices = list(np.random.permutation(len(df)))
test_indices = random.sample(permuted_indices, int(len(permuted_indices) * test_split))
train_indices = list(set(permuted_indices) - set(test_indices))

train = df.iloc[train_indices]
test = df.iloc[test_indices]

train.to_csv(data_dir + '/train.tsv', sep='\t', index=False, header=False)
test.to_csv(data_dir + '/test.tsv', sep='\t', index=False, header=True)

In [85]:
class MultiClassColaProcessor(ColaProcessor):
    def get_labels(self):
        """See base class."""
        return list([str(x) for x in labels.values()])

In [86]:
task_name = 'cola'
bert_config_file = model_dir + '/bert_config.json'
vocab_file = model_dir + '/vocab.txt'
init_checkpoint = model_dir + '/bert_model.ckpt'
do_lower_case = True
max_seq_length = 72
do_train = True
do_eval = False
do_predict = False
train_batch_size = 32
eval_batch_size = 32
predict_batch_size = 32
learning_rate = 2e-5 
num_train_epochs = 1.0
warmup_proportion = 0.1
use_tpu = False
master = None
save_checkpoints_steps = 99999999 # <----- don't want to save any checkpoints
iterations_per_loop = 1000
num_tpu_cores = 8
tpu_cluster_resolver = None

In [88]:
start = time.time()
print("--------------------------------------------------------")
print("Starting training ...")
print("--------------------------------------------------------")

bert_config = modeling.BertConfig.from_json_file(bert_config_file)

processor = MultiClassColaProcessor()
label_list = processor.get_labels()

tokenizer = tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tpu_cluster_resolver = None
is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2

run_config = tf.contrib.tpu.RunConfig(
  cluster=tpu_cluster_resolver,
  master=master,
  model_dir=output_dir,
  save_checkpoints_steps=save_checkpoints_steps,
  tpu_config=tf.contrib.tpu.TPUConfig(
      iterations_per_loop=iterations_per_loop,
      num_shards=num_tpu_cores,
      per_host_input_for_training=is_per_host))

train_examples = processor.get_train_examples(data_dir)
num_train_steps = int(len(train_examples) / train_batch_size * num_train_epochs)
num_warmup_steps = int(num_train_steps * warmup_proportion)

model_fn = model_fn_builder(
      bert_config=bert_config,
      num_labels=len(label_list),
      init_checkpoint=init_checkpoint,
      learning_rate=learning_rate,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      use_tpu=use_tpu,
      use_one_hot_embeddings=use_tpu)

estimator = tf.contrib.tpu.TPUEstimator(
      use_tpu=use_tpu,
      model_fn=model_fn,
      config=run_config,
      train_batch_size=train_batch_size)
      
      
train_file = os.path.join(output_dir, "train.tf_record")

file_based_convert_examples_to_features(
    train_examples, label_list, max_seq_length, tokenizer, train_file)

tf.logging.info("***** Running training *****")
tf.logging.info("  Num examples = %d", len(train_examples))
tf.logging.info("  Batch size = %d", train_batch_size)
tf.logging.info("  Num steps = %d", num_train_steps)

train_input_fn = file_based_input_fn_builder(
    input_file=train_file,
    seq_length=max_seq_length,
    is_training=True,
    drop_remainder=True)
    
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

end = time.time()
print("--------------------------------------------------------")
print("Training complete in ", end - start, " seconds")
print("--------------------------------------------------------")

--------------------------------------------------------
Starting training ...
--------------------------------------------------------
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


INFO:tensorflow:Using config: {'_model_dir': './output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 99999999, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_ex


Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running train on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (32, 72)
INFO:tensorflow:  name = input_mask, shape = (32, 72)
INFO:tensorflow:  name = is_real_example, shape = (32,)
INFO:tensorflow:  name = label_ids, shape = (32,)
INFO:tensorflow:  name = segment_ids, shape = (32, 72)



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (30522, 128), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 128), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 128

INFO:tensorflow:examples/sec: 219.248
INFO:tensorflow:global_step/sec: 6.02602
INFO:tensorflow:examples/sec: 192.833
INFO:tensorflow:global_step/sec: 6.71355
INFO:tensorflow:examples/sec: 214.834
INFO:tensorflow:global_step/sec: 4.23863
INFO:tensorflow:examples/sec: 135.636
INFO:tensorflow:global_step/sec: 6.89875
INFO:tensorflow:examples/sec: 220.76
INFO:tensorflow:global_step/sec: 5.86648
INFO:tensorflow:examples/sec: 187.727
INFO:tensorflow:global_step/sec: 6.06254
INFO:tensorflow:examples/sec: 194.001
INFO:tensorflow:global_step/sec: 7.3016
INFO:tensorflow:examples/sec: 233.651
INFO:tensorflow:global_step/sec: 7.09446
INFO:tensorflow:examples/sec: 227.023
INFO:tensorflow:global_step/sec: 6.62462
INFO:tensorflow:examples/sec: 211.988
INFO:tensorflow:global_step/sec: 6.06253
INFO:tensorflow:examples/sec: 194.001
INFO:tensorflow:global_step/sec: 7.17764
INFO:tensorflow:examples/sec: 229.684
INFO:tensorflow:global_step/sec: 5.76526
INFO:tensorflow:examples/sec: 184.488
INFO:tensorflow:

In [89]:
def file_based_input_fn_builder(input_file, seq_length, is_training,
                                drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  name_to_features = {
      "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
      "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "label_ids": tf.FixedLenFeature([], tf.int64),
      "is_real_example": tf.FixedLenFeature([], tf.int64),
  }

  def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.parse_single_example(record, name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
      t = example[name]
      if t.dtype == tf.int64:
        t = tf.to_int32(t)
      example[name] = t

    return example

  def input_fn(params):
    """The actual input function."""
    
    #batch_size = params["batch_size"]
    batch_size = 64 # <----- hardcoded batch_size added here 
    
    # For training, we want a lot of parallel reading and shuffling.
    # For eval, we want no shuffling and parallel reading doesn't matter.
    d = tf.data.TFRecordDataset(input_file)
    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.apply(
        tf.contrib.data.map_and_batch(
            lambda record: _decode_record(record, name_to_features),
            batch_size=batch_size,
            drop_remainder=drop_remainder))

    return d

  return input_fn

In [90]:
start = time.time()
print("--------------------------------------------------------")
print("Starting inference ...")
print("--------------------------------------------------------")

predict_examples = processor.get_test_examples(data_dir)
num_actual_predict_examples = len(predict_examples)

predict_file = os.path.join(output_dir, "predict.tf_record")

file_based_convert_examples_to_features(predict_examples, label_list,
                                        max_seq_length, tokenizer,
                                        predict_file)

tf.logging.info("***** Running prediction*****")
tf.logging.info("  Num examples = %d (%d actual, %d padding)",
                len(predict_examples), num_actual_predict_examples,
                len(predict_examples) - num_actual_predict_examples)
tf.logging.info("  Batch size = %d", predict_batch_size)

predict_drop_remainder = True if use_tpu else False
predict_input_fn = file_based_input_fn_builder(
    input_file=predict_file,
    seq_length=max_seq_length,
    is_training=False,
    drop_remainder=predict_drop_remainder)

result = estimator.predict(input_fn=predict_input_fn)

output_predict_file = os.path.join(output_dir, "test_results.tsv")

with tf.gfile.GFile(output_predict_file, "w") as writer:
    num_written_lines = 0
    tf.logging.info("***** Predict results *****")
    for (i, prediction) in enumerate(result):
        probabilities = prediction["probabilities"]
        if i >= num_actual_predict_examples:
            break
        output_line = "\t".join(
            str(class_probability)
            for class_probability in probabilities) + "\n"
        writer.write(output_line)
        num_written_lines += 1
        
end = time.time()
print("--------------------------------------------------------")
print("Inference complete in ", end - start, " seconds")
print("--------------------------------------------------------")

--------------------------------------------------------
Starting inference ...
--------------------------------------------------------
INFO:tensorflow:Writing example 0 of 408
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test-1
INFO:tensorflow:tokens: [CLS] 4 [SEP]
INFO:tensorflow:input_ids: 101 1018 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:label: 0 (id = 0)
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test-2
INFO:tensorflow:tokens: [CLS] 1 [SEP]
INFO:tensorflow:input_ids: 101 1015 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0

INFO:tensorflow:  name = output_weights:0, shape = (10, 128)
INFO:tensorflow:  name = output_bias:0, shape = (10,)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./output\model.ckpt-51
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:prediction_loop marked as finished
INFO:tensorflow:prediction_loop marked as finished
--------------------------------------------------------
Inference complete in  2.764305591583252  seconds
--------------------------------------------------------


In [91]:
labels

{'space': 0,
 'nineeleven': 1,
 'qanon': 2,
 'epstein': 3,
 'alexjones': 4,
 'covid': 5,
 'newworldorder': 6,
 'clinton': 7,
 'vaccines': 8,
 'climate': 9}